In [ ]:
import os
import numpy as np
import random
from PIL import Image
from torchvision import transforms
from collections import defaultdict
from sklearn.preprocessing import MultiLabelBinarizer
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import shutil

In [ ]:
LABELS_PATH = "path_to_labels"
IMAGE_PATH = "path_to_images"  

In [3]:
# Görüntü–etiket eşlemesini hazırla
image_ids = []
labels_per_image = []

for fn in os.listdir(IMAGE_PATH):
    if not fn.lower().endswith((".jpg",".png")): continue
    img_id = os.path.splitext(fn)[0]
    lb_path = os.path.join(LABELS_PATH, img_id + ".txt")
    classes = set()
    if os.path.isfile(lb_path):
        with open(lb_path) as f:
            for line in f:
                cls = int(line.split()[0])
                classes.add(cls)
    image_ids.append(img_id)
    labels_per_image.append(list(classes))

print(image_ids)
print(labels_per_image)


['acar_1', 'acar_100', 'acar_101', 'acar_102', 'acar_103', 'acar_105', 'acar_11', 'acar_111', 'acar_113', 'acar_116', 'acar_119', 'acar_12', 'acar_120', 'acar_125', 'acar_126', 'acar_13', 'acar_14', 'acar_15', 'acar_16', 'acar_17', 'acar_19', 'acar_20', 'acar_21', 'acar_22', 'acar_23', 'acar_24', 'acar_241', 'acar_25', 'acar_250', 'acar_251', 'acar_252', 'acar_253', 'acar_254', 'acar_255', 'acar_256', 'acar_257', 'acar_258', 'acar_259', 'acar_26', 'acar_260', 'acar_261', 'acar_262', 'acar_263', 'acar_264', 'acar_265', 'acar_266', 'acar_267', 'acar_268', 'acar_269', 'acar_270', 'acar_271', 'acar_272', 'acar_273', 'acar_274', 'acar_275', 'acar_276', 'acar_277', 'acar_278', 'acar_279', 'acar_280', 'acar_281', 'acar_282', 'acar_283', 'acar_284', 'acar_285', 'acar_286', 'acar_287', 'acar_288', 'acar_289', 'acar_29', 'acar_290', 'acar_291', 'acar_292', 'acar_293', 'acar_294', 'acar_295', 'acar_296', 'acar_297', 'acar_298', 'acar_299', 'acar_30', 'acar_300', 'acar_301', 'acar_302', 'acar_303'

In [ ]:
#toplam sınıf sayısı:
NUM_CLASSES = 26

mlb = MultiLabelBinarizer()
labels_matrix = mlb.fit_transform(labels_per_image)
# sonuç: (n_samples, NUM_CLASSES) şeklinde 0/1 dizisi

print("Binarizer sınıfları:", mlb.classes_)

Binarizer sınıfları: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


In [4]:
import os

def make_split_dirs(base_dir="stra_dataset_split"):
    splits = ["train", "val", "test"]
    subs   = ["images", "labels"]
    for split in splits:
        for sub in subs:
            path = os.path.join(base_dir, split, sub)
            os.makedirs(path, exist_ok=True)
    print(f"Directories created under '{base_dir}'")

# Usage
make_split_dirs("stra_dataset_split")


Directories created under 'stra_dataset_split'


In [10]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import shutil

import os
import shutil
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

def multilabel_split(image_ids, labels_matrix, out_dir,
                              random_state=42):
    os.makedirs(out_dir, exist_ok=True)

    # 1) İlk önce train = %70, rem = %30
    msss1 = MultilabelStratifiedShuffleSplit(
        n_splits=1, test_size=0.30, random_state=random_state
    )
    train_idx, rem_idx = next(msss1.split(image_ids, labels_matrix))

    # 2) Kalan %30’un içinden %50’si val, %50’si test  ➔ her biri %15 total
    image_ids_rem   = [image_ids[i] for i in rem_idx]
    labels_rem      = labels_matrix[rem_idx]  # eğer numpy array ise direkt bu; list ise [labels_matrix[i] for i in rem_idx]
    msss2 = MultilabelStratifiedShuffleSplit(
        n_splits=1, test_size=0.50, random_state=random_state
    )
    val_rel_idx, test_rel_idx = next(msss2.split(image_ids_rem, labels_rem))

    # 3) Orijinal indekslere geri çevir
    train_ids = train_idx
    val_ids   = [rem_idx[i] for i in val_rel_idx]
    test_ids  = [rem_idx[i] for i in test_rel_idx]

    # 4) Klasörleri oluştur ve dosyaları kopyala
    for split_name, idxs in (("train", train_ids),
                             ("val",   val_ids),
                             ("test",  test_ids)):
        for i in idxs:
            img_id = image_ids[i]
            for ext in (".jpg", ".png"):
                src_img = os.path.join(IMAGE_PATH, img_id + ext)
                if not os.path.isfile(src_img):
                    continue
                dst_img = os.path.join(out_dir, split_name, "images")
                dst_lb  = os.path.join(out_dir, split_name, "labels")
                os.makedirs(dst_img, exist_ok=True)
                os.makedirs(dst_lb, exist_ok=True)
                shutil.copy(src_img, os.path.join(dst_img, img_id + ext))
                shutil.copy(os.path.join(LABELS_PATH, img_id + ".txt"),
                            os.path.join(dst_lb,  img_id + ".txt"))

    print("Split tamam! train/val/test =", 
          len(train_ids), len(val_ids), len(test_ids))


In [ ]:
multilabel_split(
    image_ids=image_ids,
    labels_matrix=labels_matrix,        # or labels_per_image if you adapted it
    out_dir= "path_to_output_dir"
)

Split tamam! train/val/test = 8005 1709 1739


In [ ]:
from collections import Counter
import glob

lbl_dir = "path_to_labels_dir"
counts = Counter()
label_paths = glob.glob(f"{lbl_dir}/*.txt")
for lp in label_paths:
    for line in open(lp):
        cls = int(line.split()[0])
        counts[cls] += 1
max_count = max(counts.values())


In [13]:
aug_needed = {cls: max_count - n for cls, n in counts.items() if n < max_count}
aug_needed

{7: 264,
 20: 736,
 24: 973,
 9: 510,
 19: 198,
 18: 49,
 23: 267,
 4: 323,
 25: 690,
 13: 527,
 21: 804,
 15: 455,
 6: 870,
 5: 890,
 3: 480,
 0: 1006,
 2: 957,
 11: 1072,
 1: 1055,
 8: 1075,
 10: 1079,
 14: 1017,
 12: 1053,
 22: 1087,
 17: 1071}

In [14]:
# augment_per_image[cls] = her orijinal görüntüden kaç tane augment üretilecek
import math
augment_per_image = {}
for cls, need in aug_needed.items():
    orig = counts[cls]
    augment_per_image[cls] = math.ceil(need / orig)

In [ ]:
import cv2, os
import os
import cv2
import yaml
import albumentations as A
import glob
 
img_dir = "path_to_images_dir"
 
 # Pipeline
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.3),
    A.Rotate(limit=5, p=0.3),
    A.Blur(blur_limit=3, p=0.2),
])
 
# Klasörler
os.makedirs("augmented/images", exist_ok=True)
os.makedirs("augmented/labels", exist_ok=True)
  
# minority artık sadece counts[cls]<max_count olan sınıfları içeriyor
minority = set(aug_needed.keys())
 
for lbl_path in label_paths:
    img_path = os.path.join(img_dir, os.path.basename(lbl_path)[:-4] + ".jpg")
    labels = [int(l.split()[0]) for l in open(lbl_path)]
    # bu görüntü hangi minority sınıfları içeriyor?
    this_minority = [c for c in labels if c in minority]
    if not this_minority:
        continue
 
    EXTS = ['.jpg', '.jpeg', '.png']
 
    # Önce None atayalım, sonra bulursak oku
    img = None
    base = os.path.basename(lbl_path)[:-4]
    for ext in EXTS:
        candidate = os.path.join(img_dir, base + ext)
        if os.path.isfile(candidate):
            img = cv2.imread(candidate)
            img_path = candidate
            break
 
    if img is None:
        print(f"[WARN] Hiçbir uzantı bulunamadı: {base} -> {[base+e for e in EXTS]}")
        continue
    # en çok augmenta ihtiyacı olan sınıf kadar üret
    n_aug = max(augment_per_image[c] for c in this_minority)
    for i in range(n_aug):
        aug = transform(image=img)["image"]
        cv2.imwrite(f"augmented/images/{base}_aug{i}.jpg", aug)
        open(f"augmented/labels/{base}_aug{i}.txt","w").write(open(lbl_path).read())